# BioData Catalyst Data Release QA
Validation tests in this notebook:
1. [**Patient counts of new studies**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-New-study-patient-counts): Patient counts of the new studies from the integration environment are compared to the patient counts in Patient_Count_Per_Consents.csv
2. [**Data dictionary comparison**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-Data-dictionary-comparison): Integration and production data dictionaries are compared to ensure complete match

### Prerequisites
- Developer access to the integration enviroment (token)
- Consent value(s) of the new study (or studies) to validate (phs number)
- Knowledge on whether a harmonized study was added

### Install packages

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import json
from pprint import pprint

import pandas as pd
import math

from shutil import copyfile
import PicSureClient
import PicSureBdcAdapter

### Connect to PIC-SURE
Be sure to use the **developer token** from the **integration environment**. It is necessary to have access to all studies to validate the counts.

In [ ]:
integration_PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
int_resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
integration_token_file = "token.txt"

In [ ]:
with open(integration_token_file, "r") as f:
    my_int_token = f.read()

In [ ]:
int_client = PicSureClient.Client()
int_connection = int_client.connect(integration_PICSURE_network_URL, my_int_token, True)
int_adapter = PicSureBdcAdapter.Adapter(int_connection)
int_resource = int_adapter.useResource(int_resource_id)

## Validation: New study patient counts
The purpose of this section is to validate the patient counts for newly ingested studies.

### Specify the new study (or studies) to be tested
To validate the new studies ingested, specify the phs numbers in the cell below without the consent group. 
For example, if the study of interest is the AMISH study, list `phs000956` below (*not* `phs000956.c1`).

In [ ]:
#to_validate = ['list', 'phs_numbers', 'here']
to_validate = ['phs000285', #CARDIA,
               'phs000703', #CATHGEN
               'phs001194', #PCGC
               'phs000810', #HCHS-SOL
               'phs001252' #ECLIPSE
              ]
to_validate_harmonized = ['phs000285', #CARDIA
                          'phs000810' #HCHS-SOL
                         ]

to_validate_topmed = []

### Get patient count file from S3 bucket
This notebook uses `Patient_Count_Per_Consents.csv` from the S3 bucket as the reference file. First we need to copy this file over to this directory.

In [ ]:
src = '/home/ec2-user/SageMaker/studies/ALL-avillach-73-bdcatalyst-etl/general/completed/Patient_Count_Per_Consents.csv'
dst = '/home/ec2-user/SageMaker/biodatacatalyst-pic-sure/access-dashboard-metadata/Patient_Count_Per_Consents.csv'
copyfile(src, dst)

In [ ]:
# Load S3 file as a dataframe in the Jupyter Notebook
patient_ref_file = pd.read_csv('Patient_Count_Per_Consents.csv', header=None, names=['consent', 'patient_count'])
patient_ref_file

In [ ]:
# Extract the consent groups based on the user-identified phs values
full_phs = []
for phs in to_validate:
    for full in list(patient_ref_file['consent']):
        if phs in full and ".c0" not in full:
            full_phs.append(full)
full_phs

### Get patient count for the specified consent groups
Now the consent groups will be used to find the patient counts currently in the integration environment.

In [ ]:
# Start a new query and initialize output dictionary
patient_count_query = int_resource.query()
output = {}

# Get patient counts for each consent group
for consentGroup in full_phs:
    print(consentGroup)
    patient_count_query.filter().delete("\\_consents\\") # Delete all consents
    patient_count_query.filter().add("\\_consents\\", consentGroup) # Add back consent group of interest
    patient_count = patient_count_query.getCount() # Get patient count
    output[consentGroup] = patient_count # Add to output

### Compare the values to the reference file
Finally we will compare the values from the reference file to the counts in the integration environment.

In [ ]:
for consent_val in output.keys():
    ref_count = int(patient_ref_file[patient_ref_file['consent'] == consent_val]['patient_count']) # Count from reference file
    integration_count = output[consent_val] # Count from integration environment
    # Display result message
    if ref_count == integration_count:
        print(consent_val, "passes validation")
    else:
        print('***DID NOT PASS VALIDATION:', consent_val)
        print('Expected count from Patient_Count_Per_Consents.csv:\t', ref_count)
        print('Count retrieved from integration environment:\t', integration_count)

## Validation: Data dictionary comparison
The purpose of this section is to compare the data dictionaries of the production and integration environments. These data dictionaries should be identical besides the studies that are being loaded and/or updated.

### Establish connection to production environment

In [ ]:
production_PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
prod_resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
production_token_file = "prod_token.txt"

In [ ]:
with open(production_token_file, "r") as f:
    my_prod_token = f.read()

In [ ]:
prod_client = PicSureClient.Client()
prod_connection = prod_client.connect(production_PICSURE_network_URL, my_prod_token, True)
prod_adapter = PicSureBdcAdapter.Adapter(prod_connection)
prod_resource = prod_adapter.useResource(prod_resource_id)

### Load data dictionaries
Next we will load the data dictionaries from the integration and production environments as dataframes. These will be used to compare the environments.

In [ ]:
int_dictionary = int_resource.dictionary()
prod_dictionary = prod_resource.dictionary()

In [ ]:
integ = int_dictionary.find().DataFrame()

In [ ]:
prod = prod_dictionary.find().DataFrame()

### Compare dictionaries
The following comparisons will be made between the dictionaries:
1. Find concept paths that exist in integration, but not production
2. Find concept paths that exist in production, but not integration
3. Identify differences in the dataframe between integration and production

The first comparisons use the concept paths, which we will extract and compare now.

In [ ]:
integration = list(integ.index)
production = list(prod.index)

In [ ]:
# FIRST COMPARISON
# In integration, but not production
diff = list(set(integration)-set(production))

# Filter out the newly loaded studies
wrong = []
for i in diff:
    if not any(study in i for study in to_validate):
        wrong.append(i)

if len(wrong)<1:
    print("All concepts in production are in integration.")
else:
    print("The following concepts are in integration but not production:")
    for i in wrong:
        print(i)

In [ ]:
# SECOND COMPARISON
# In production, but not integration
diff2 = list(set(production)-set(integration))

if len(diff2)<1:
    print("All concepts in integration are in production.")
else:
    print("The following concepts are in production but not in integration:")
    for i in diff2:
        print(i)

The third comparison compares the data in the data dictionary. The following function iterates through each row of the data dictionary and compares the integration and production results.

In [ ]:
# THIRD COMPARISON
# Initializing variables used in function
columns = ['min', 'max', 
           'patientCount', 'observationCount', 
           'categorical', 'HpdsDataType', 
           'categoryValues', 'description'] # Names of columns in data dictionary
to_check = {} # Results dictionary

# Genomic variables 
genome_vars = ['Variant_consequence_calculated', 
               'Gene_with_variant', 
               'Variant_class', 'Variant_frequency_as_text', 
               'Variant_severity']
genome_keys = ['patientCount', 'observationCount']

Specify if the harmonized concept paths should be compared. If a harmonized study was added, set `harmonized_added = True`. If a harmonized study was not added, set `harmonized_added = False`.

This information is used to improve the comparison process. If a harmonized study was added, we expect all `DCC Harmonized data set` concept paths to be different. So, in this scenario, these concepts paths do not need to be compared.

In [ ]:
# Change to true if harmonized were added, false if not.
harmonized_added = True

*Please note these cells take a while to run.*

In [ ]:
# Compare each row and column of the integration and production data dictionaries


for integ_key in integration:
    # Skip the concept paths in integration and not in production (defined above as 'diff')
    if integ_key not in diff:
        # Skip harmonized concept paths if harmonized study was added
        if harmonized_added == True and '\\DCC Harmonized data set\\' in integ_key:
            continue
            '''This can be improved - we would know the expected difference between prod and int and can check this'''
            '''Do this later'''
        mini_list = []
        # Iterate through columns of data dictionary
        for i in columns:
            # Account for instances where min and max could be nan
            if i in ['min', 'max']:
                isitnan1 = math.isnan(integ.loc[integ_key][i])
                isitnan2 = math.isnan(prod.loc[integ_key][i])
                if isitnan1 and isitnan2: # Test if both are nan - if so, pass match test
                    test = True
                else: # If not nan, test if values match
                    test = integ.loc[integ_key][i] == prod.loc[integ_key][i]
                    
            # Genomic variables are recorded differently - this section accounts for nans
            elif i in genome_keys:
                # Account for unique instances
                if integ_key in genome_vars: # Check if both are nans
                    test = math.isnan(integ.loc[integ_key][i]) == math.isnan(prod.loc[integ_key][i])
                else: # Check if values match
                    test = integ.loc[integ_key][i] == prod.loc[integ_key][i]
                    
            # Account for all other rows
            else:
                test = integ.loc[integ_key][i] == prod.loc[integ_key][i]
            
            # Record each column that doesn't pass validation
            if test == False:
                mini_list.append(i)
                #print(integ_key, '\n', i) # Option to print rows and columns that do not match
        if len(mini_list) > 0:
            to_check[integ_key] = mini_list

In [ ]:
# View differing results
misc_consents = ['\\_consents\\', '\\_studies_consents\\', 
                 '\\_Parent Study Accession with Subject ID\\', 
                 '\\_parent_consents\\'] # Concept paths that we expect to be different with newly loaded studies

# Can account for the patient count differences using Patientcountperconsents.csv
# Make edits to account for these differences in the consent concept paths

'''Can also check the observationCount between the consents concept paths - more complicated, do this later'''

if harmonized_added == True:
    misc_consents.append('\\_harmonized_consent\\')
    
for key in to_check.keys():
    if key not in misc_consents:
        print("\nConcept path that differs in data dictionaries:")
        print(key)
        print("\nColumns that do not match:")
        for obs in to_check[key]:
            print(obs)
            print("\tIntegration value:", integ.loc[key][obs])
            print("\tProduction value:", prod.loc[key][obs])
print("\nCheck complete.")

In [ ]:
# Accounting for differences in patient counts
harmonized_phs = []
for phs in to_validate_harmonized:
    for full in list(patient_ref_file['consent']):
        if phs in full and ".c0" not in full:
            harmonized_phs.append(full)

# Total number of patients in newly loaded studies
diff_total = patient_ref_file[patient_ref_file['consent'].isin(full_phs)].sum()['patient_count']
harmonized_diff_total = patient_ref_file[patient_ref_file['consent'].isin(harmonized_phs)].sum()['patient_count']

for key in to_check.keys():
    if key in misc_consents:
        print("\nConcept path that differs in data dictionaries:")
        print(key)
        #print("\nColumns that do not match:")
        for obs in to_check[key]:
            if obs != 'categoryValues':
                if integ.loc[key][obs]-diff_total != prod.loc[key][obs] and integ.loc[key][obs]-harmonized_diff_total != prod.loc[key][obs]:
                    print(obs)
                    print("\tIntegration value:", integ.loc[key][obs])
                    print("\tProduction value:", prod.loc[key][obs])
                else:
                    print('\t', obs, 'differs as expected.')

In [ ]:
to_check

In [ ]:
prod.loc['\\_consents\\']['patientCount']

In [ ]:
integ.loc['\\_studies_consents\\']['patientCount']

In [ ]:
prod.loc['\\_studies_consents\\']['patientCount']

In [ ]:
loaded_count_diff = 0
for i in patient_ref_file['consent']:
    if i in full_phs:
        print(patient_ref_file[patient_ref_file['consent']==i]['patient_count'])
        loaded_count_diff += patient_ref_file[patient_ref_file['consent']==i]['patient_count']
loaded_count_diff

In [ ]:
diff_total = patient_ref_file[patient_ref_file['consent'].isin(full_phs)].sum()['patient_count']

In [ ]:
diff_total